In [14]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [38]:
SIA = SentimentIntensityAnalyzer()

In [ ]:
SIA.polarity_sores("Hello")

## Load & inspect datasets

Dataset

In [3]:
df_clean_pmc = pd.read_csv("data/clean_pmc.csv", skiprows=0, header=None)
df_clean_noncomm_use = pd.read_csv("data/clean_noncomm_use.csv", skiprows=1, header=None)
df_clean_comm_use = pd.read_csv("data/clean_comm_use.csv", skiprows=1, header=None)
df_biorxiv_clean = pd.read_csv("data/biorxiv_clean.csv", skiprows=1, header=None)

In [4]:
df_concat = pd.concat([df_clean_pmc, df_clean_comm_use, df_clean_noncomm_use, df_biorxiv_clean])

In [5]:
data = df_concat.to_numpy()

In [6]:
data.shape

(13203, 9)

In [7]:
data[0]

array(['paper_id', 'title', 'authors', 'affiliations', 'abstract', 'text',
       'bibliography', 'raw_authors', 'raw_bibliography'], dtype=object)

In [8]:
df_drug_ratings = pd.read_csv("data/drugsComTest_raw.csv", skiprows=0, header=None)

In [9]:
drug_ratings = df_drug_ratings.to_numpy()

In [10]:
drug_ratings.shape

(53767, 7)

In [11]:
drug_ratings[1]

array(['163740', 'Mirtazapine', 'Depression',
       '"I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I&#039;ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."',
       '10', '28-Feb-12', '22'], dtype=object)

In [12]:
drugs = np.hstack([np.unique(drug_ratings[1:,1]), np.array(['chloroquine'])])
conditions = np.unique(drug_ratings[1:,2].astype(str))[44:]

In [13]:
print(drugs.shape, conditions.shape)

(2638,) (665,)


In [ ]:
documents = data[1:, 5].tolist()
sentiments = []
print("total", len(documents))
for i, d in enumerate(documents):
    if i%10 == 0: print("progress", i, end="\r")
    documents[i] = word_tokenize(d.lower())
    sentiments.append(SIA.polarity_scores(d))

total 13202


In [17]:
wordcounts = []
for i, d in enumerate(documents):
    wordcounts.append(Counter(d))

In [18]:
outs = []
for drug in drugs:
    num = 0
    for i, w in enumerate(wordcounts):
        num += sentiments[i]w[drug.lower()]
    outs.append((num, drug))

In [60]:
sorted(outs, reverse=True)

[(3348, 'Glucose'),
 (2035, 'Ribavirin'),
 (1755, 'Oseltamivir'),
 (1186, 'Urea'),
 (1123, 'chloroquine'),
 (1122, 'Lysine'),
 (908, 'Silver'),
 (631, 'Adenosine'),
 (588, 'Luminal'),
 (587, 'Tetracycline'),
 (577, 'Zanamivir'),
 (558, 'Copper'),
 (540, 'Amantadine'),
 (450, 'Tryptophan'),
 (441, 'Doxycycline'),
 (433, 'Biotin'),
 (420, 'Gentamicin'),
 (388, 'Azithromycin'),
 (370, 'Phenol'),
 (350, 'Dexamethasone'),
 (327, 'Fluoride'),
 (321, 'Doxorubicin'),
 (300, 'Aspirin'),
 (288, 'Prednisolone'),
 (282, 'Vancomycin'),
 (265, 'Ketamine'),
 (257, 'Amoxicillin'),
 (253, 'Acyclovir'),
 (250, 'Methylprednisolone'),
 (208, 'Propofol'),
 (206, 'Cyclosporine'),
 (204, 'Cetuximab'),
 (202, 'Ciprofloxacin'),
 (196, 'Imiquimod'),
 (185, 'Tamoxifen'),
 (185, 'Acidophilus'),
 (184, 'Imatinib'),
 (182, 'Metronidazole'),
 (173, 'Prednisone'),
 (171, 'Rosiglitazone'),
 (171, 'Midazolam'),
 (171, 'Hydrocortisone'),
 (170, 'Montelukast'),
 (168, 'Erythromycin'),
 (158, 'Caffeine'),
 (157, 'Echinace